# Helical Torsion Spring

<table><tr>
<td>
<img src="./img/Helical_Torsion_Spring/Torsion_Spring.png" width=330"/> </td>
<td>
<img src="./img/Helical_Torsion_Spring/Torsion_Spring_-_End_Types.png" width=200"/> </td>
</tr></table>

Importing the relevant Classes

In [2]:
from me_toolbox.springs import HelicalTorsionSpring, Spring
from me_toolbox.tools import Nmm_per_rad_to_Nmm_per_deg

## Geometric properties

d - Wire diameter<br>
D - Spring mean diameter<br>
l1 - First leg<br>
l2 - second leg

In [3]:
d = 1.829  # [mm]
OD = 15.081  # [mm]
D = OD - d  
print(f"{D = :.2f}")
pin = 10.16  # [mm]
l1 = 25.4  # [mm]
l2 = l1  # [mm]

D = 13.25


## Material properties

Table 10-4 from Shigley:
<table><tr>
<td>
<img src="./img/Helical_Compression_Spring/Tensile_strength_estimation.png" width=360"/> </td>
<td>
<img src="./img/Helical_Extension_Spring/Mechanical_properties_of_some_spring_wires_-_table_10-5.png" width=300"/> </td>
</tr></table>

Material options for use in Spring.material_prop method are:<br>
  * music wire<br>
  * oq&t wire<br>
  * hard-drawn wire<br>
  * chrome-vanadium wire<br>
  * chrome-silicon wire<br>
  * 302 stainless wire<br>
  * phosphore-bronze wire<br>

G - Shear modulus<br>
E - Elastic modulus<br>
Sut - Ultimate tensile strength

In [4]:
# music wire
G = 81e3  # [MPa]
E = 196.5e3  # [MPa]
Sut = Spring.material_prop('music wire', d, metric=True, verbose=False)
print(f"Sut = {Sut:.2f}")

Sut = 2025.67


Table 10-6 from Shigley:
<table><tr>
<td> <img src="./img/Helical_Compression_Spring/present_of_tensile_strength.png" width=600"/> </td>
</tr>
</table>

><b>Note: </b>For Torsion springs we only use the values from the first column devided by <b>0.577</b>


In [5]:
yield_percent = 0.45 / 0.577
print(f"yield_percent = {yield_percent:.2}")

yield_percent = 0.78


## Forces and spring rate

k - Spring rate<br>
Tmax - Maximum Torque<br>
Tmin - Minimum Torque

In [6]:
k = 525.11  # [Nmm/rad]
Tmax = 851.27

><b>Note:</b> For torsion spring the spring rate is $\Large\frac{N \cdot mm}{rad}$
and that is what our class uses but some times $\Large\frac{N \cdot mm}{deg}$ is used
so I added a conversion function

In [7]:
print(f"k={Nmm_per_rad_to_Nmm_per_deg(k):.4f} [Nmm/deg]")

k=9.1649 [Nmm/deg]


## Defining the spring

Required parameters:
  * max_moment - The maximum torque in $[Nmm]$
  * wire_diameter - The wire diameter in $[mm]$
  * spring_diameter - The spring diameter in $[mm]$
  * leg1 - Effective length of the first spring's legs $[mm]$
  * leg2 - Effective length of the second spring's legs $[mm]$
  * ultimate_tensile_strength - The ultimate tensile strength in $[MPa]$
  * yield_precent - the precentage of the shear yield strength(Ssy) from the ultimate tensile strength (Sut)
  * shear_modulus - The material shear modulus in $[MPa]$
  * elastic_modulus - The material elastic modulus in $[MPa]$
  * end_type - The shape of the ends of the spring (used to calculate number of coils)
  * spring_rate - The spring rate in $[Nmm/rad]$

Optional parameters: <br>
(the bold parameters are the defaults)

  * arbor_diameter=**None** - The arbor size (Torsion springs usually used with a rod for support) $[mm]$
  * shot_peened=**False** - If the spring underwent shot peening
  * density=**None** - The Material density in $[kg/mm^3]$

In [8]:
spring = HelicalTorsionSpring(max_moment=Tmax,
                              wire_diameter=d,
                              spring_diameter=D,
                              leg1=l1,
                              leg2=l2,
                              ultimate_tensile_strength=Sut,
                              yield_percent=yield_percent,
                              shear_modulus=G,
                              elastic_modulus=E,
                              spring_rate=k,
                              arbor_diameter=pin,
                              shot_peened=False,
                              density=7800)

The clearance between the spring and arbor after tension is applied is negative (-2.159049722447463)


### Calculated geometric properties

In [9]:
print(f"Free Length (L0) = {spring.free_length:.2f} mm\n"
      f"Active Coils (Na) = {spring.active_coils:.2f}\n"
      f"Total Coils (Nb) = {spring.body_coils:.2f}\n"
      f"Partial turn (β) = {spring.partial_turn*360:.2f} degree")

Free Length (L0) = 7.77 mm
Active Coils (Na) = 4.66
Total Coils (Nb) = 4.25
Partial turn (β) = 32400.77 degree


#### Change in length and diameter caused by the applied load

In [10]:
print(f"loaded Length (L') = {spring.loaded_length:.2f} mm\n"
      f"loaded Diameter (D') = {spring.loaded_diameter:.2f}mm")

loaded Length (L') = 10.06 mm
loaded Diameter (D') = 9.83mm


### Calculated material properties

In [11]:
print(f"Shear yield strength (Sy) = {spring.yield_strength:.2f}")

Shear yield strength (Sy) = 1579.81


## Static analysis

In [12]:
print(f"static safety factor = {spring.static_analysis():.2f}")

static safety factor = 1.00


## Fatigue analysis
<table><tr>
<td> <img src="./img/Helical_Torsion_Spring/Table_10-10_-_ maximum_allowable_stresses_in_cyclic_applications.png" width=600"/> </td>
</tr>
</table>

> Note: The verbose option can be used to print more information

In [13]:
Tmax = 564.896
Tmin = 112.979
nf, ns, N, Sf = spring.fatigue_analysis(max_moment=Tmax, min_moment=Tmin, fatigue_percent=53, criterion='gerber', reliability=50, verbose=False)
print(f"Fatigue safety factor (nf) = {nf:.2f}\n"
      f"Static safety factor (ns) = {ns:.2f}\n"
      f"Number of cycles untill failure (N) = {N}\n"
      f"Stress at failure (Sf) = {Sf}")

Fatigue safety factor (nf) = 1.78
Static safety factor (ns) = 1.51
Number of cycles untill failure (N) = inf
Stress at failure (Sf) = None




## Frequency analysis

TBD